In [0]:
"""!pip3 install 'torch==1.3.1'
!pip3 install 'torchvision==0.5.0'
!pip3 install 'Pillow-SIMD'
!pip3 install 'tqdm'"""
# !pip install --upgrade wandb

In [0]:
import os
import logging

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn

import torchvision
from torchvision import transforms
#from torchvision.models import alexnet # resnet18, resnet34

from PIL import Image
from tqdm import tqdm
import random


import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import time

In [0]:
DEVICE = 'cuda' # 'cuda' or 'cpu'
DATA_DIR = 'DATA' # here the dataset will be downloaded

NUM_CLASSES = 100 

BATCH_SIZE = 128     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch size, learning rate should change by the same factor to have comparable results

LR = 0.01           # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5  # Regularization, you can keep this at the default

NUM_EPOCHS = 70      # Total number of training epochs (iterations over dataset)
STEP_SIZE = 49       # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 10

dataset

In [0]:
# Clone github repository with dataset handler
!rm -r Cifar100/
!rm -r $DATA_DIR
!mkdir "DATA"
if not os.path.isdir('./Cifar100'):
  !git clone https://github.com/danielegenta/Progetto-MLDL.git
  !mv 'Progetto-MLDL' 'Cifar100'
  !rm -r Cifar100/Theoretical-Sources
  !rm -rf Cifar100/ProjectMLDL.ipynb

In [0]:
# Download dataset from the official source and save it into DATA/cifar-100-pyhton

if not os.path.isdir('./{}'.format("$DATA_DIR/cifar-100-python")):
    !wget https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz
    !tar -xf 'cifar-100-python.tar.gz'  
    !mkdir $DATA_DIR
    !mv 'cifar-100-python' "$DATA_DIR/cifar-100-python"
    !rm -rf 'cifar-100-python.tar.gz'

In [0]:
# Define transforms for training phase
train_transform = transforms.Compose([
                                      transforms.Pad(4),         # Add padding
                                      transforms.RandomCrop(32), # Crops a random squares of the image  
                                      transforms.ToTensor(), 
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) 
                                      ])
# Define transforms for the evaluation phase
eval_transform = transforms.Compose([transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))                                 
])

In [0]:
from Cifar100.Dataset.cifar100 import CIFAR100


# Import dataset
train_dataset = CIFAR100(DATA_DIR, split='train', transform=train_transform)
test_dataset = CIFAR100(DATA_DIR, split='test', transform=eval_transform)

# check if datasets have been correctly loaded
print(len(train_dataset))
print(len(test_dataset))

In [0]:
from Cifar100.reverse_index import ReverseIndex

def build_test_splits(dataset, reverse_index):
    splits = dict()
    groups = list(reverse_index.getGroups())
    for g in groups:
        labels_of_groups = reverse_index.getLabelsOfGroup(g)
        indices = list(dataset.df[dataset.df['labels'].isin(labels_of_groups)].index)
        splits[g] = indices
    return splits

In [0]:
# Dataloaders iterate over pytorch datasets and transparently provide useful functions (e.g. parallelization and shuffling)
train_subsets = []
val_subsets = []
test_subsets = []

for v in train_splits.values():
    train_subs = Subset(train_dataset, v['train'])
    val_subs = Subset(train_dataset, v['val'])
    train_subsets.append(train_subs)
    val_subsets.append(val_subs)

for i in range(0,10):
    v=test_splits[i]
    test_subs = Subset(test_dataset, v)
    test_subsets.append(test_subs)

In [0]:
# to upd
from Cifar100.resnet import resnet34

def getResNet34(output_size):
    net = resnet34(num_classes=output_size)
    # net.fc = nn.Linear(net.fc.in_features, output_size) # embedded in the class

    criterion = nn.CrossEntropyLoss()
    parameters_to_optimize = net.parameters()
    optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

    return net, criterion, optimizer, scheduler

def addOutputs(net, num):
    net.addOutputNodes(num)

def getNet():
    return getResNet34(10)

def getSchedulerOptimizer(net):
    parameters_to_optimize = net.parameters()
    optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)
    return optimizer, scheduler

**Exemplars management**<br>
From iCaRL. We have an exemplar set for each class that we have seen so far. The cardinality of each exemplar set is constant and it is equal, at any time, to m = K/t. Where K is a constraint equal to the amount of memory we're allocating for the exemplars and t is the number of classes that has been seen so far. Implementing iCaRL, whenever a group of (10) classes is trained, it is trained on the train data for those classes (as before) + the current exemplars sets.*italicized text*

In [0]:
# default params

K = 2000
total_classes = 100
classes_group = 10

from cifar100.icarl-model import ICaRL

# to upd
icarl = ICaRL(2048, 1)
#icarl.cuda() # is it useful?

In [0]:
from PIL import Image
from torch.autograd import Variable


def incrementalTraining(getNet, addOutputs, train_subsets, val_subsets, test_subsets, K):
    
    net, criterion, optimizer, scheduler = getNet() #should this be updated?????????
    train_set = None
    test_set = None

    current_train_num = 0
    total_trains = len(train_subsets)

    # exemplars new params
    classes_seen = 0 # number of classes seen so far (useful in the computation of the exemplar sets cardinality)
    #tot_num_classes = 100 

    for train_subset, val_subset, test_subset in zip(train_subsets, val_subsets, test_subsets):
        #  phase_start = time.time()
        current_train_num += 1

        if train_set is None:
            train_set = train_subset
        else:
            train_set = joinSubsets(train_dataset, [train_set, train_subset])

        optimizer, scheduler = getSchedulerOptimizer(net) # Should this be updated????????

        train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=False)
        val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, drop_last=False)
        test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, drop_last=False)

        ####### EXEMPLARS NEW CODE (following alg. 2 on icarl paper) ##################
        # compute the cardinality of each exemplar set
        
        # 1 - update representation of the net 
        # @todo

        # 2 - update m (number of images per class in the exemplar set corresponding to that class)
        m = K/icarl.n_classes

        # 3 - reduce exemplar set for all the previously seen classes
        icarl.reduce_exemplar_sets(m)

        # retrieve the 10 classes in the current subset
        classes_current_subset = list(train_dataset.df.loc[train_subset.indices, 'labels'].value_counts().index)
        
        # 4 - construct the exemplar set for the new classes
        for y in classes_current_subset: # for each class in the current subset
          print("Constructing exemplar set for class-%d..." %(y))
          
          # extract all the imgs in the train subset that are linked to this class
          images_current_class = train_subset.dataset.df.loc[train_dataset.df['labels'] == y, 'data'] #they're TENSORS NOT IMAGES (the conversion will be done later)
          
          # compone the exemplar set
          icarl.construct_exemplar_set(images_current_class,m, eval_transorm) # why eval? ref: https://github.com/donlee90/icarl/blob/master/main.py

        # update the num classes seen so far
        icarl.n_known = icarl.n_classes #n_classes is incremented in 1: updateRepresentation

        # start classifier ....

    

K = 2000 # total amount of memory we allocate for exemplars
#exemplars(getNet, addOutputs, train_subsets, val_subsets, test_subsets, K)


In [0]:
        #############################################

        # Builds growing train and test set. The new sets include data from previous class groups and current class group
        """"
        if train_set is None:
              train_set = train_subset
          else:
              train_set = joinSubsets(train_dataset, [train_set, train_subset])
          if test_set is None:
              test_set = test_subset
          else:
              test_set = joinSubsets(test_dataset, [test_set, test_subset])

          if first_pass:
              first_pass = False
          else:
              addOutputs(net, 10)

          # Trains model on previous and current class groups
          optimizer, scheduler = getSchedulerOptimizer(net)
          train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=False)
          train(net, train_loader, criterion, optimizer, scheduler)

          # Validate model on current class group
          val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, drop_last=False)
          v_acc, v_loss = validate(net, val_loader, criterion)
          print('\nValidation accuracy: {} - Validation loss: {}\n'.format(v_acc, v_loss))

          # Test the model on previous and current class groups
          test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, drop_last=False)
          t_acc = test(net, test_loader)
          print('\nTest accuracy: {}\n'.format(t_acc))
          wandb.log({ 'Classes': current_train_num*10, 'Validation accuracy': v_acc, 'Validation loss': v_loss, 'Test accuracy': t_acc })

          print('\n\nPhase completed in {} seconds\n\n'.format(time.time() - phase_start))
          """"